**Action Required:** Update the `project_id` and `project_access_token` in order to access your data assets.

In [ ]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id={project-id}, project_access_token={access-token})
pc = project.project_context

# Watson Assistant Delete Logs

This notebook performs log deletion on the user log records of Watson Assistant.

### Table of Contents
* [1.0 Configuration and Log Collection](#config)
    * [1.3 Delete Logs](#delete-logs) <---- Click here to jump to delete logs

## Housekeeping <a class="anchor" id="housekeeping"></a>
This section will import libraries and dependencies for this notebook. 

In [ ]:
!curl -O https://raw.githubusercontent.com/cognitive-catalyst/WA-Testing-Tool/master/log_analytics/extractConversations.py
!curl -O https://raw.githubusercontent.com/cognitive-catalyst/WA-Testing-Tool/master/log_analytics/getAllLogs.py

In [ ]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter("ignore")

!pip install ibm-watson

import json
import pandas as pd
import getAllLogs
import extractConversations
from ibm_watson import AssistantV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

## 1.0 Configuration and log collection <a class="anchor" id="config"></a>
The next few cells require some configuration.  Review the variables and update them for your specific assistant.  The comments in the cells guide you in the configuration.

### 1.1 Retrieve logs from the Watson Assistant instance
This option will allow you to retrieve the logs from the Assistant API.

**Action Required:**
- Update `wa-apikey` to your Watson Assistant API key
- Update `wa-url` to your Watson Assistant URL
- Update `workspace_id` or `assistant_id` to reflect the settings in your bot
- Update `log_filter` for any other filters, e.g. update `response_timestamp` if you wish to limit the amount of data retrieved.

In [ ]:
# API, URL,and workspace ID are extractable from "View API Details page"
iam_apikey='wa-apikey' # Update this

#url pattern depends on region and when it was created (update one to match your instance)
#url="https://api.us-east.assistant.watson.cloud.ibm.com"
url='wa-url'

workspace_id=None # Update or set to None without quotes

# Filter API is described at: https://cloud.ibm.com/docs/assistant?topic=assistant-filter-reference#filter-reference
log_filter="language::en,response_timestamp>=2020-06-01" \
+",request.context.system.assistant_id::<<assistant_id>>" # If using this, uncomment and replace <<assistant_id>>

#Change the number of logs retrieved, default settings will return 100,000 logs (200 pages of 500)
page_size_limit=500
page_num_limit=200

#WA API version
version="2018-09-20" 

rawLogsJson = getAllLogs.getLogs(iam_apikey, url, workspace_id, log_filter, page_size_limit, page_num_limit, version)
rawLogsPath= custName + "_logs.json"

print('\nSaved log data to {}'.format(rawLogsPath))

### 1.2 Format logs
Update these fields by following the instructions in the comments.

In [ ]:
# Define the conversation corrrelation field name for your Watson Assistant records.
# Provide the field name as it appears in the log payload (default is 'response.context.conversation_id')
primaryLogKey = "response.context.conversation_id"
conversationKey='conversation_id'
customFieldNames = "response.context.language,response.context.metadata.user_id"

allLogsDF = extractConversations.extractConversationData(rawLogsJson, primaryLogKey, customFieldNames)
conversationsGroup = allLogsDF.groupby(conversationKey,as_index=False)

print("Total log events:",len(allLogsDF))
allLogsDF.head()

### 1.3 Delete logs from the Watson Assistant instance <a class="anchor" id="delete-logs"></a>

#### Directions: You *must* run all of the above cells before runnin the delete logs cells. To do so, click inside the top cell in this notebook then click "Shift+Enter" to run that cell. You can hold down "Shift" while tapping "Enter" to run multiple cells.

In [ ]:
authenticator = IAMAuthenticator(iam_apikey)

assistant = AssistantV1(
    version='2020-04-01',
    authenticator = authenticator
)

assistant.set_service_url(url)

In [ ]:
# Replace "1999" with the string that you would like to remove.
# This will remove all conversations associated with the user who entered that exact phrase.
removeDF = allLogsDF[allLogsDF['input.text'].str.contains("1999")]
logsToDelete = removeDF["metadata.user_id"]

if len(logsToDelete) == 0:
    print("No matches found")
else:
    print("Matches found: ")
    print(removeDF[["input.text", "metadata.user_id"]])
    print()

In [ ]:
# Only run this if you are comfortable deleting the conversations output from the above cell
# A response of {} indicates that the log has been queued for deletion
# Please note that deletion make take up to 24 hours to process

for index, value in logsToDelete.items():
        print(f"Deleting conversation: {value}")
        response = assistant.delete_user_data(
            customer_id = value
        ).get_result()

        print(json.dumps(response, indent=2))